In [2]:
import pandas as pd
from pydantic import BaseModel, field_validator
from datetime import datetime
from sqlalchemy import create_engine
from typing import Optional
import json
import logging
from pathlib import Path
from datetime import datetime
from sqlalchemy import text

now = datetime.now()

In [17]:
#Tek seferde tüm parquet yükleniyor; milyonlarca satır için chunked/stream okumaya ihtiyaç var
df = pd.read_parquet("//Users//sdedeoglu//Desktop//python//case_data.parquet.gzip")

In [ ]:
#connection
path = Path("//Users//sdedeoglu//Desktop//python//config.json").read_text(encoding="utf-8")
config = json.loads(path) 

kullanici = config['kullanici']
sifre = config['sifre']  
host = config['host']   
port = config['port']  
veritabani = config['veritabani'] 
engine = create_engine(f"mysql+pymysql://{kullanici}:{sifre}@{host}:{port}/{veritabani}")

In [19]:
df.dropna(subset=["user_id", "session_id", "hotel_id", "request_id", "funnel_id"], inplace=True)

 #Pydantic 

@field_validator("alan1", "alan2", mode="before") — alan değerleri Pydantic model oluşturulmadan önce bu fonksiyondan geçirilir.
mode="after" — önce Pydantic tipi uygulanır, sonra validator çalışır.
Birden fazla alanı aynı validator ile hedefleyebilirsin.
Döndürülen değer, o alanın nihai değeri olur; hata fırlatılırsa validation hatası oluşur.

In [8]:
true_set = {"yes", "true", "1", "y", "evet"}
false_set = {"no", "false", "0", "n", "hayir", "hayır"}

formats = [
"%Y-%m-%d %H:%M:%S.%f",  
"%Y-%m-%d %H:%M:%S",     
"%Y-%m-%dT%H:%M:%S.%fZ", 
"%Y-%m-%dT%H:%M:%S",     
"%d/%m/%Y %H:%M:%S",     
"%d-%m-%Y %H:%M:%S"]

status = {
"done": "success",
"ok": "success",
"paid": "success",
"fail": "failed",
"error": "failed"}

TABLE_PK_MAP = {
    "users": ["user_id"],
    "sessions": ["user_id", "session_id"],
    "events": ["request_id", "session_id", "hotel_id", "funnel_id"],
    "hotels": ["hotel_id"],
    "payments": ["request_id"]}

In [21]:
class UserModel(BaseModel):
    #veri tiplerinin belirlenmesi
    user_id: Optional[float]
    subscriber_id: Optional[float]
    country: str
    has_email_contact_permission: Optional[bool]
    has_phone_contact_permission: Optional[bool]

    @field_validator("has_email_contact_permission", "has_phone_contact_permission", mode="before")
    def to_bool(cls, v):
        # izin sutunlarinin bool tipine cevrilmesi - daha deterministik davranis
        if v is None:
            return None
        if isinstance(v, bool):
            return v
        if isinstance(v, (int, float)):
            # 0 -> False, diğerleri True (int/float için net davranış)
            try:
                return bool(int(v))
            except Exception:
                raise ValueError(f"Boolean parse failed for numeric value: {v}")
        if isinstance(v, str):
            s = v.strip().lower()
            if s in true_set:
                return True
            if s in false_set:
                return False
        raise ValueError(f"Cannot parse boolean value: {v}")

In [22]:
class EventModel(BaseModel):
    request_id: str
    session_id: str
    funnel_id: str
    hotel_id: int
    timestamp: datetime
    page_name: str
    search_query: Optional[str]
    destination_id: Optional[float]
    num_guests: Optional[float]
    #timestamp sutununda belirli formatlarda gelen tarih degerlerini datetime objesine cevirme
    @field_validator("timestamp", mode="before")
    def parse_timestamp(cls, v):
        if isinstance(v, datetime):
            return v
        if v is None:
            raise ValueError("timestamp boş olamaz")
        if isinstance(v, str):
            s = v.strip()
            # her formati dener; ilk başarılı parse döndürülür
            for fmt in formats:
                try:
                    return datetime.strptime(s, fmt)
                except ValueError:
                    continue
            raise ValueError("timestamp geçersiz")
        raise ValueError(f"Unsupported timestamp type: {type(v)}")

In [23]:
class HotelModel(BaseModel):
    hotel_id: int
    hotel_price: Optional[float]
    currency: str
#otel ucretlerinde gereksiz karakterlerin temizlenmesi 
    @field_validator("hotel_price", mode="before")
    def clean_price(cls, v):
        if v is None:
            return None
        if isinstance(v, str):
            v = v.replace(",", ".").replace("$", "").strip()
        try:
            return float(v)
        except (ValueError, TypeError):
            return None

In [24]:
class PaymentModel(BaseModel):
    request_id: str
    payment_status: Optional[str]
    confirmation_number: Optional[str]
#odeme sutununun belirlenen fortmata cevrilmesi
    @field_validator("payment_status", mode="before")
    def normalize_status(cls, v):
        if v is None:
            return None
        # string harici gelirse string'e çevirip normalize et
        if not isinstance(v, str):
            v = str(v)
        v = v.strip().lower()
        return status.get(v, v)

#icerik kontrolu (after)
    @field_validator("payment_status", mode="after")
    def check_valid_values(cls, v):
        allowed = {"pending", "success", "failed"}
        if v is None:
            return None
        if v not in allowed:
            raise ValueError(f"Invalid payment status: {v}")
        return v

In [25]:
class SessionModel(BaseModel):
    session_id: str
    user_id: Optional[float]
    user_agent: Optional[str]
    device_type: Optional[str]
    ip_address: Optional[str]
    utm_source: Optional[str]

    @field_validator("session_id", mode="before")
    def session_id_not_empty(cls, v):
        if v is None:
            raise ValueError("session_id boş olamaz")
        if isinstance(v, str):
            v = v.strip()
            if not v:
                raise ValueError("session_id boş olamaz")
            return v
        return str(v)

In [ ]:
#Validation & Normalization

def validate_with_pydantic(df):
    users, sessions, events, hotels, payments = [], [], [], [], []

    for row in df.to_dict(orient="records"):
        try:
            users.append(UserModel(**row).model_dump(exclude_none=True))
            sessions.append(SessionModel(**row).model_dump(exclude_none=True))
            events.append(EventModel(**row).model_dump(exclude_none=True))
            hotels.append(HotelModel(**row).model_dump(exclude_none=True))
            payments.append(PaymentModel(**row).model_dump(exclude_none=True))
        except Exception as e:
            print(f"Validation error: {e}")

    return (
        pd.DataFrame(users).drop_duplicates(subset=['user_id']),
        pd.DataFrame(sessions).drop_duplicates(subset=['session_id']),
        pd.DataFrame(events),
        pd.DataFrame(hotels).drop_duplicates(subset=['hotel_id']),
        pd.DataFrame(payments)
    )

users, sessions, events, hotels, payments = validate_with_pydantic(df)

In [ ]:
users["updated_Date"] = now
sessions["updated_Date"] = now
events["updated_Date"] = now
hotels["updated_Date"] = now
payments["updated_Date"] = now

In [ ]:
def upsert_df(engine, df, table_name, chunk_size=1000):
    # Tablo PK'leri al
    pk_cols = TABLE_PK_MAP.get(table_name)
    cols = list(df.columns)
    cols_quoted = ", ".join(f"`{c}`" for c in cols)
    vals_placeholders = ", ".join(":" + c for c in cols)
    
    # UPDATE kısmı
    update_parts = []
    for c in cols:
        if c in pk_cols:
            continue
        if c == "updated_Date":
            update_parts.append(f"`{c}` = GREATEST(`{c}`, VALUES(`{c}`))")  #eğer tabloda o PK değeri yoksa direkt olarak insert yapılıyor.
        else:
            #sadece tarih daha yeni ve gelen sütun değeri mevcut değerden farklı ise overwrite olur; tarih farklı ama değerler aynıysa hiçbir sütun değişmez.
            update_parts.append(f"`{c}` = IF(VALUES(`updated_Date`) > `updated_Date` AND NOT (VALUES(`{c}`) <=> `{c}`), VALUES(`{c}`), `{c}`)")

    update_clause = ", ".join(update_parts) if update_parts else "/* no-update */"
    sql = f"""INSERT INTO `{table_name}` ({cols_quoted}) VALUES ({vals_placeholders}) 
              ON DUPLICATE KEY UPDATE {update_clause}"""

    records = df.to_dict(orient="records")
    if not records:
        return

    with engine.begin() as conn:
        for i in range(0, len(records), chunk_size):
            chunk = records[i:i+chunk_size]
            conn.execute(text(sql), chunk)

In [ ]:
upsert_df(engine, users, "users")
upsert_df(engine, sessions, "sessions")
upsert_df(engine, events, "events")
upsert_df(engine, hotels, "hotels")
upsert_df(engine, payments, "payments")